In [3]:
import cv2
import os, copy
import numpy as np
import torch
import torch.nn as nn
import sys
import torchvision.transforms as tr
from model import UNet, WPU_Net
from tools import posprecess, stitch, proprecess_img, tailor, get_expansion
import matplotlib.pyplot as plt
import torch.nn.functional as F
import time
from weight_map_loss import caculate_weight_map

import pdb, sys
cwd = os.getcwd()
print(cwd)

# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

transform = tr.Compose([
    tr.ToTensor(),
    tr.Normalize(mean = [ 0.9336267],    # RGB
                std = [ 0.1365774])
])

cwd = os.getcwd()
output_test_dir = os.path.join(cwd, "../datasets", "segmentation", "net_test", "test")
output_test_crop_dir = os.path.join(cwd, "../datasets", "segmentation", "net_test", "test_overlap_crop")
output_train_dir = os.path.join(cwd, "../datasets", "segmentation", "net_train", "train")
output_train_crop_dir = os.path.join(cwd, "../datasets", "segmentation", "net_train", "train_crop")
output_val_dir = os.path.join(cwd, "../datasets", "segmentation", "net_train", "val")
output_val_crop_dir = os.path.join(cwd, "../datasets", "segmentation", "net_train", "val_crop")

name = 'WPU_Net_model'

model_path = "./parameter/"+name+"/best_model_state.pth"
result_save_dir = "./result/"+name+"/"
result_total_save_dir = "./result_total/"+name+"/"

if not os.path.exists(result_save_dir):
    os.makedirs(result_save_dir)
    
if not os.path.exists(result_total_save_dir):
    os.makedirs(result_total_save_dir)
    

D:\WPU-Net\segmentation


In [2]:
model = WPU_Net(num_channels=2, multi_layer=True) 

if torch.cuda.is_available():
    model = nn.DataParallel(model).cuda()
    
model.load_state_dict(torch.load(model_path))

model.eval()

multi_layer  True


DataParallel(
  (module): WPU_Net(
    (down1): Conv3x3(
      (conv1): Sequential(
        (0): Conv2d(2, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
    (down2): Conv3x3(
      (conv1): Sequential(
        (0): Conv2d(65, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
   

In [4]:
%%time

import glob

DATA_FOLDER = output_test_crop_dir
images = sorted(os.listdir(os.path.join(DATA_FOLDER, "images")))
print(len(images))

start_time = time.time()
count = 0

min_file = 149   # 1  117  149
max_file = 296   # 116  148  296

for item in images:
    if item.endswith(".png"):
        filename = item.split(".")[0]
        pic_num = item.split("_")[0]
        if int(pic_num) > min_file and int(pic_num) <= max_file:
            count += 1
            
            name = filename

            test_image = os.path.join(DATA_FOLDER, "images/"+name+".png")
            img = proprecess_img(test_image)
            
            # WPU-Net
            last_name = str(int(pic_num) - 1).zfill(3) + '_' + filename.split('_')[1] + '_' + filename.split('_')[2]
            last_mask = cv2.imread(os.path.join(DATA_FOLDER, "labels/"+last_name+".png"), 0)

            last_tensor = torch.Tensor(np.array(last_mask)).unsqueeze(0).unsqueeze(0)
            
#             # last mask
            last_tensor[last_tensor == 255] = -6
            last_tensor[last_tensor == 0] = 1
        
            output = model(inputs = img, last = last_tensor)

            result_npy = posprecess(output, close=True)
            
            cv2.imwrite(os.path.join(result_save_dir, filename+".png"), result_npy)
            
end_time = time.time()
average_time = (end_time - start_time)/count
print("end ...", average_time)

DATA_FOLDER = output_test_dir
imgList = sorted(os.listdir(os.path.join(DATA_FOLDER, "images")))
print(len(imgList))

n = 0
for img in imgList:
    if img.endswith(".png"):
        name = img.split(".")[0]
        print(name)
        if int(name) > min_file and int(name) <= max_file:
            stitch(256, 256, name, result_save_dir, result_total_save_dir+name+".png", 32)
            n += 1
            print(name, " ", n)
print("end...")

80
end ... 0.13544202223420143
5
149
150
3   3
150   1
151
3   3
151   2
152
3   3
152   3
153
3   3
153   4
end...
Wall time: 9.16 s


In [6]:
# evaluate  

from evaluation import eval_RI_VI, eval_F_mapKaggle

RI_save_dir = "./evaluations/big_RI_VI/"
Map_save_dir = "./evaluations/big_F_mAP/"

if not os.path.exists(RI_save_dir):
    os.makedirs(RI_save_dir)
    
if not os.path.exists(Map_save_dir):
    os.makedirs(Map_save_dir)
    
print("WPU_Net_model " + "#####"*20)
eval_RI_VI("./result_total/WPU_Net_model/", RI_save_dir+"WPU_Net_model.txt")
eval_F_mapKaggle("./result_total/WPU_Net_model/", Map_save_dir+"WPU_Net_model.txt")